<a href="https://colab.research.google.com/github/Farii01/Internship_PINN_work/blob/main/Updated_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git

  Cloning https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git to /tmp/pip-req-build-jrpxbtl4
  Running command git clone --filter=blob:none --quiet https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git /tmp/pip-req-build-jrpxbtl4
  Resolved https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git to commit 87d5eab82dbae0c55a8d93113cd4ac6db38a0bf0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import pygeotools
import torch
import torch.nn as nn
import numpy

In [3]:
model_path = "/content/COVOBS-x2_400reals.hdf5"

pygeo = pygeotools.pygeotools()
model_name=  "COVOBS-x2_400reals.hdf5"
pygeo.loadModel(
    modelName="COVOBS-x2_400reals.hdf5",
    modelType="covobs_hdf5",
    modelPath=model_path
)

pygeo.isLoaded("COVOBS-x2_400reals.hdf5")  # Should return True

pygeotools was initialized with `verbose=True`.


True

### Retrieving the data

In [4]:
# Setting the grid
pygeo.setGrid("1deg")

# Creating the context
context = {
    "lmax": 13,
    "r": pygeo.constants["rCore"]
}

In [5]:
# Computing the MF and SV
MF = pygeo.addMeasure("COVOBS-x2_400reals.hdf5", "MF", context)
SV = pygeo.addMeasure("COVOBS-x2_400reals.hdf5", "SV", context)

13


In [6]:
# Retrieving the grid
_, (thetas, phis) = pygeo.getCurrentGrid()

## Generating the patches for input in PINN

we cut the Earth into small square patches, train a separate neural network on each patch, then stitch the results together.


In [7]:
def generate_patches(theta_size=20, phi_size=20, overlap=5):# 20 rows longitude, 20 cols latitude
    patches = []
# without overlap
    dtheta = theta_size - overlap
    dphi = phi_size - overlap

#going from pole to pole (0 to 180 in colatitude) by stepping in 15° chunks (because dtheta = 15)
    for i1 in range(0, 180 - theta_size + 1, dtheta):
        i2 = i1 + theta_size
        for j1 in range(0, 360 - phi_size + 1, dphi):
            j2 = j1 + phi_size

            patches.append((i1, i2, j1, j2))

    return patches
'''
This loop creates a full list of (i1, i2, j1, j2) — patches of size 20°×20°, sliding across the whole globe, overlapping 5°.

Each patch will be used to train a small PINN, then all will be stitched together into one global map.
'''
# How to use?
patches = generate_patches(theta_size=20, phi_size=20, overlap=5)

for i1, i2, j1, j2 in patches:
    ... # For each patch, we have to solve the inverse problem

In [8]:
# Selecting the data
Br = pygeo.selectFromMeasure("COVOBS-x2_400reals.hdf5", MF, options={"component": "r", "time": 2020})
dBrdt = pygeo.selectFromMeasure("COVOBS-x2_400reals.hdf5", SV, options={"component": "r", "time": 2020})
'''
#Take a small patch (20° × 20°)
i1, i2 = 60, 80
j1, j2 = 330, 350

# Computing the derivative wrt theta and phi of the MF
dBrdth = numpy.gradient(Br, thetas, axis=0)[i1:i2,j1:j2,...]
dBrdph = numpy.gradient(Br, phis, axis=1)[i1:i2,j1:j2,...]

Br = Br[i1:i2,j1:j2,...]
dBrdt = dBrdt[i1:i2,j1:j2,...]

# Slicing the grid
thetas_bis = thetas[i1:i2]
phis_bis = phis[j1:j2]

'''

'\n#Take a small patch (20° × 20°)\ni1, i2 = 60, 80\nj1, j2 = 330, 350\n\n# Computing the derivative wrt theta and phi of the MF\ndBrdth = numpy.gradient(Br, thetas, axis=0)[i1:i2,j1:j2,...]\ndBrdph = numpy.gradient(Br, phis, axis=1)[i1:i2,j1:j2,...]\n\nBr = Br[i1:i2,j1:j2,...]\ndBrdt = dBrdt[i1:i2,j1:j2,...]\n\n# Slicing the grid\nthetas_bis = thetas[i1:i2]\nphis_bis = phis[j1:j2]\n\n'

### Creating the NN

In [9]:
node_inputs = 2
node_outputs = 2
node_layer = 64
hidden_layers = 3

In [10]:
#  Defining the NN
# For now, it has one hidden layer with 32 nodes
# The activation functions are TANH
class CoreFlowPINN(nn.Module):
    def __init__(self):
        super(CoreFlowPINN, self).__init__()

        layers = []

        layers.append(nn.Linear(node_inputs, node_layer))
        layers.append(nn.Tanh())

        for _ in range(hidden_layers):
            layers.append(nn.Linear(node_layer, node_layer))
            layers.append(nn.Tanh())

        layers.append(nn.Linear(node_layer, node_outputs))

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [11]:
'''
# Shaping everything for the NN

# Creating the grid
thetas_grid, phis_grid = numpy.meshgrid(thetas_bis, phis_bis, indexing="ij")

# Creating the (flatten) grid points
thetas_flatten = thetas_grid.flatten()
phis_flatten = phis_grid.flatten()

# Creating grid tensors to feed the NN
thetas_nn = torch.tensor(thetas_flatten[:, None], dtype=torch.float32, requires_grad=True)
phis_nn = torch.tensor(phis_flatten[:, None], dtype=torch.float32, requires_grad=True)

# Creating tensors for the MF and SV
Br_nn = torch.tensor(Br.flatten()[:, None], dtype=torch.float32)
dBrdt_nn = torch.tensor(dBrdt.flatten()[:, None], dtype=torch.float32)
dBrdth_nn = torch.tensor(dBrdth.flatten()[:, None], dtype=torch.float32)
dBrdph_nn = torch.tensor(dBrdph.flatten()[:, None], dtype=torch.float32)

# Ravel the angles
inputs = torch.cat([thetas_nn, phis_nn], dim=1)
'''

'\n# Shaping everything for the NN\n\n# Creating the grid\nthetas_grid, phis_grid = numpy.meshgrid(thetas_bis, phis_bis, indexing="ij")\n\n# Creating the (flatten) grid points\nthetas_flatten = thetas_grid.flatten()\nphis_flatten = phis_grid.flatten()\n\n# Creating grid tensors to feed the NN\nthetas_nn = torch.tensor(thetas_flatten[:, None], dtype=torch.float32, requires_grad=True)\nphis_nn = torch.tensor(phis_flatten[:, None], dtype=torch.float32, requires_grad=True)\n\n# Creating tensors for the MF and SV\nBr_nn = torch.tensor(Br.flatten()[:, None], dtype=torch.float32)\ndBrdt_nn = torch.tensor(dBrdt.flatten()[:, None], dtype=torch.float32)\ndBrdth_nn = torch.tensor(dBrdth.flatten()[:, None], dtype=torch.float32)\ndBrdph_nn = torch.tensor(dBrdph.flatten()[:, None], dtype=torch.float32)\n\n# Ravel the angles\ninputs = torch.cat([thetas_nn, phis_nn], dim=1)\n'

In [21]:
print(f"Number of patches: {len(patches)}")


Number of patches: 253


### Adding the physics

In [12]:
"""
We want to solve the radial induction equation at the CMB
    dBr / dt + ∇h • (Uh Br) = 0

with some quasi-geostrophic condition (meaning the flow will partly align along the axis of rotation, making columns-like flow)
    ∇h • (Uh cos(θ)) = 0

As the flow is incompressible, it is subject to ∇ • U = 0. As a result, it admits a unique toroidal-poloidal decomposition:
    Uh = ∇ x (r T) + ∇ (r S)

Thus, instead of directly predicting uθ and uφ, it appears clever to predict T and S as it already enforces the incompressibility condition.

In spherical coordinates, one has
    uθ = -(dT/dφ) / sin(θ) + dS/dθ
    uφ = dT/dθ + (dS/dφ) / sin(θ)
"""

r = torch.tensor(pygeo.constants["rCore"]) # placing ourselves at the CMB

def compute_loss(model, inputs, thetas_nn, phis_nn, Br_nn, dBrdt_nn, dBrdth_nn, dBrdph_nn):

    # Retrieving the predicted flow
    u_pred = model(inputs)

    # Retrieving the toroidal and poloidal components

    T = u_pred[:, 0:1]
    S = u_pred[:, 1:2]

    # First derivatives of T and S
    dT_dth = torch.autograd.grad(T, thetas_nn, grad_outputs=torch.ones_like(T), create_graph=True, retain_graph=True)[0]
    dT_dph = torch.autograd.grad(T, phis_nn, grad_outputs=torch.ones_like(T), create_graph=True, retain_graph=True)[0]
    dS_dth = torch.autograd.grad(S, thetas_nn, grad_outputs=torch.ones_like(S), create_graph=True, retain_graph=True)[0]
    dS_dph = torch.autograd.grad(S, phis_nn, grad_outputs=torch.ones_like(S), create_graph=True, retain_graph=True)[0]

    """
    Computing L1
    """
    # Computing the L1² loss function
    # L1² = || dBr / dt + ∇h • (Uh Br) ||²
    # ∇h • (Uh Br) = (∇h • Uh) Br + Uh • (∇h Br)

    sin_th = torch.sin(thetas_nn)
    cos_th = torch.cos(thetas_nn)
    tan_th = torch.tan(thetas_nn)

    # We are defining u_th and u_ph with T and S
    u_th = -dT_dph / sin_th + dS_dth
    u_ph = dT_dth + dS_dph / sin_th

    # Computing ∇h • Uh
    u_th_sin_th = u_th * sin_th
    d_u_th_sin_th_dth = torch.autograd.grad(u_th_sin_th, thetas_nn, grad_outputs=torch.ones_like(u_th_sin_th), create_graph=True, retain_graph=True)[0]

    d_u_ph_dph = torch.autograd.grad(u_ph, phis_nn, grad_outputs=torch.ones_like(u_ph), create_graph=True, retain_graph=True)[0]

    divH_uH = (1 / (r * sin_th)) * (d_u_th_sin_th_dth + d_u_ph_dph)
    # divH_uH = u_th * tan_th / r

    # Computing ∇h Br
    # The derivatives are provided as they are not the NN variables but inputs
    gradH_Br_th = (1 / r) * dBrdth_nn
    gradH_Br_ph = (1 / (r * sin_th)) * dBrdph_nn

    # Wrapping the induction equation
    L1 = dBrdt_nn + Br_nn * divH_uH + u_th * gradH_Br_th + u_ph * gradH_Br_ph

    """
    Computing L2
    """
    L2 = divH_uH - u_th * tan_th / r

    return L1, L2

In [ ]:
# I moved everything to the training loop so that for each patch in every iteration, the model receives the correct,
# patch-specific input like thetas_nn, phis_nn, Br_nn, dBrdt_nn, dBrdth_nn, dBrdph_nn

# Training over all patches
num_realizations = 5 # 5 time on each patch
epochs = 1000
λ = 1000

loss_grids = []
patch_centers = []

for i1, i2, j1, j2 in patches:
    if i1 < 10 or i2 > 170 or (i1 <= 90 <= i2):
        continue  # skip poles and equator

    Br_patch = Br[i1:i2, j1:j2, ...]
    dBrdt_patch = dBrdt[i1:i2, j1:j2, ...]
    dBrdth = numpy.gradient(Br, thetas, axis=0)[i1:i2, j1:j2, ...]
    dBrdph = numpy.gradient(Br, phis, axis=1)[i1:i2, j1:j2, ...]

    thetas_bis = thetas[i1:i2]
    phis_bis = phis[j1:j2]

    thetas_grid, phis_grid = numpy.meshgrid(thetas_bis, phis_bis, indexing="ij")
    thetas_flat = thetas_grid.flatten()
    phis_flat = phis_grid.flatten()

    thetas_nn = torch.tensor(thetas_flat[:, None], dtype=torch.float32, requires_grad=True)
    phis_nn = torch.tensor(phis_flat[:, None], dtype=torch.float32, requires_grad=True)
    Br_nn = torch.tensor(Br_patch.flatten()[:, None], dtype=torch.float32)
    dBrdt_nn = torch.tensor(dBrdt_patch.flatten()[:, None], dtype=torch.float32)
    dBrdth_nn = torch.tensor(dBrdth.flatten()[:, None], dtype=torch.float32)
    dBrdph_nn = torch.tensor(dBrdph.flatten()[:, None], dtype=torch.float32)

    inputs = torch.cat([thetas_nn, phis_nn], dim=1)

    loss_history = []

    for run in range(num_realizations):
        print(f"\n=== Training run {run + 1}/{num_realizations} ===")
        model = CoreFlowPINN()
        optimizer = torch.optim.Adam(model.parameters(), 0.001)

        for epoch in range(epochs):
            optimizer.zero_grad()
            L1, L2 = compute_loss(model, inputs, thetas_nn, phis_nn, Br_nn, dBrdt_nn, dBrdth_nn, dBrdph_nn)
            L1_loss = (L1**2).mean()
            L2_loss = (L2**2).mean()
            Loss = L1_loss + λ * L2_loss
            Loss.backward()

            optimizer.step()

            if epoch % 500 == 0:
                print(f"Epoch {epoch:4d} | L1 Loss: {L1_loss.item():12.6e} | L2 Loss: {L2_loss.item():12.6e} | Total Loss: {Loss.item():12.6e}", flush=True)
'''

Later for graphs
                L1_final, _ = compute_loss(model, inputs, thetas_nn, phis_nn, Br_nn, dBrdt_nn, dBrdth_nn, dBrdph_nn)
                L1_squared = (L1_final**2).detach().cpu().numpy().flatten()
                loss_history.append(L1_squared)

    mean_loss_map = numpy.mean(loss_history, axis=0)
    loss_grid = mean_loss_map.reshape(len(thetas_bis), len(phis_bis))

    loss_grids.append(loss_grid)
    patch_centers.append(((i1 + i2) // 2, (j1 + j2) // 2))
'''
print(" Done computing for all patches.")




=== Training run 1/5 ===
Epoch    0 | L1 Loss: 1.327360e+06 | L2 Loss: 1.058399e-10 | Total Loss: 1.327360e+06
Epoch  500 | L1 Loss: 6.835668e+02 | L2 Loss: 1.178818e-05 | Total Loss: 6.835786e+02

=== Training run 2/5 ===
Epoch    0 | L1 Loss: 1.289237e+06 | L2 Loss: 2.340753e-10 | Total Loss: 1.289237e+06
Epoch  500 | L1 Loss: 7.785200e+02 | L2 Loss: 1.123165e-05 | Total Loss: 7.785312e+02

=== Training run 3/5 ===
Epoch    0 | L1 Loss: 1.288331e+06 | L2 Loss: 5.249894e-10 | Total Loss: 1.288331e+06
Epoch  500 | L1 Loss: 5.944352e+02 | L2 Loss: 8.806851e-06 | Total Loss: 5.944440e+02

=== Training run 4/5 ===
Epoch    0 | L1 Loss: 1.407890e+06 | L2 Loss: 1.438577e-09 | Total Loss: 1.407890e+06
Epoch  500 | L1 Loss: 6.158219e+02 | L2 Loss: 1.022188e-05 | Total Loss: 6.158321e+02

=== Training run 5/5 ===
Epoch    0 | L1 Loss: 1.329882e+06 | L2 Loss: 1.014578e-11 | Total Loss: 1.329882e+06
Epoch  500 | L1 Loss: 6.654930e+02 | L2 Loss: 7.135322e-06 | Total Loss: 6.655001e+02

=== Train